# Hacker stats I

In [1]:
import numpy as np
import pandas as pd

import iqplot

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [8]:
df = pd.read_csv('data/grant_complete.csv', comment='#')

df.head()

,band,beak depth (mm),beak length (mm),species,year
0,20123,8.05,9.25,fortis,1973
1,20126,10.45,11.35,fortis,1973
2,20128,9.55,10.15,fortis,1973
3,20129,8.75,9.95,fortis,1973
4,20133,10.15,11.55,fortis,1973


In [9]:
df = df.loc[
    (df["species"] == "scandens") & (df["year"].isin([1975, 2012])),
    ["year", "beak depth (mm)"],
].reset_index(drop=True)

df.head()

,year,beak depth (mm)
0,1975,8.4
1,1975,8.8
2,1975,8.4
3,1975,8.0
4,1975,7.9


In [10]:
p = iqplot.ecdf(
    data=df,
    q='beak depth (mm)',
    cats='year',
)

bokeh.io.show(p)

In [14]:
# Pull our data sets as Numpy arrays
bd_1975 = df.loc[df['year']==1975, 'beak depth (mm)'].values
bd_2012 = df.loc[df['year']==2012, 'beak depth (mm)'].values

# Compute the means
np.mean(bd_1975), np.mean(bd_2012)

(8.959999999999999, 9.188492063492063)

In [21]:
rng = np.random.default_rng()
bs_sample = rng.choice(bd_1975, replace=True, size=len(bd_1975)) 

In [22]:
bs_sample

array([ 9.  ,  8.8 ,  9.1 ,  8.9 ,  9.74,  8.6 ,  8.6 , 10.1 ,  8.4 ,
        8.1 ,  9.8 ,  8.35,  9.6 ,  9.3 ,  8.35,  9.6 , 10.4 ,  9.1 ,
        9.1 ,  8.9 ,  9.45,  9.45,  9.45,  8.7 ,  9.7 ,  9.7 ,  8.7 ,
        7.9 ,  9.45,  9.  ,  9.7 ,  8.8 ,  9.7 , 10.4 ,  9.65,  8.6 ,
        8.9 ,  8.8 ,  8.9 ,  9.1 ,  9.3 ,  8.8 ,  9.1 ,  8.2 ,  9.2 ,
        8.8 ,  8.7 ,  9.6 ,  9.2 ,  9.45,  8.6 ,  8.6 ,  8.4 ,  9.45,
        8.6 ,  9.8 ,  8.5 ,  8.5 ,  9.05, 10.4 ,  8.5 ,  9.45,  9.1 ,
        9.45,  9.1 ,  9.8 ,  9.1 ,  9.  ,  9.2 ,  8.3 ,  8.9 ,  9.1 ,
        9.  ,  9.1 ,  8.1 , 10.1 , 10.1 , 10.4 ,  9.  ,  8.4 ,  8.3 ,
        8.  ,  8.4 ,  9.8 ,  9.45,  8.3 ,  8.8 ])

In [24]:
bs_replicate = np.mean(bs_sample)
bs_replicate

9.08494252873563

In [25]:
def bs_sample(data):
    return rng.choice(data, replace=True, size=len(data))

def bs_replicate(data, func, args=()):
    return func(bs_sample(data), *args)

In [38]:
n_reps = 100_000
bs_reps_1975 = np.array([bs_replicate(bd_1975, np.mean) for _ in range(n_reps)])

In [39]:
bs_reps_1975

array([8.94632184, 9.00827586, 9.03068966, ..., 8.93229885, 9.01333333,
       8.94344828])

In [40]:
np.percentile(bs_reps_1975, [2.5, 97.5])

array([8.84333333, 9.07965517])

In [41]:
conf_int_1975 = np.percentile(bs_reps_1975, [2.5, 97.5])
conf_int_1975

array([8.84333333, 9.07965517])

In [42]:
# Initialize bootstrap replicas array
bs_reps_1975 = np.empty(n_reps)

# Compute replicates
for i in range(n_reps):
    bs_reps_1975 = draw_bs_rep(bd_1975, np.mean, rng)

NameError: name 'draw_bs_rep' is not defined

In [43]:
def draw_bs_rep(data, func, rng):
    """Compute a bootstrap replicate from data."""
    bs_sample = rng.choice(data, size=len(data))
    return func(bs_sample)

In [44]:
# Initialize bootstrap replicas array
bs_reps_1975 = np.empty(n_reps)

# Compute replicates
for i in range(n_reps):
    bs_reps_1975 = draw_bs_rep(bd_1975, np.mean, rng)

In [46]:
bs_reps_1975 = np.array(
    [draw_bs_rep(bd_1975, np.mean, rng) for _ in range(n_reps)]
)

In [47]:
# Compute replicates
bs_reps_2012 = np.array(
    [draw_bs_rep(bd_2012, np.mean, rng) for _ in range(n_reps)]
)

# Compute the confidence interval
conf_int_2012 = np.percentile(bs_reps_2012, [2.5, 97.5])

In [48]:
print(conf_int_1975)
print(conf_int_2012)

[8.84333333 9.07965517]
[9.07301587 9.30595238]


In [49]:
years = ['2012', '1975']
p = bokeh.plotting.figure(
    frame_height=100,
    frame_width=250,
    x_axis_label='beak depth (mm)',
    y_range=years,
)

p.circle([bd_2012.mean(), bd_1975.mean()], years, size=5)
p.line(conf_int_1975, ['1975']*2, line_width=3)
p.line(conf_int_2012, ['2012']*2, line_width=3)

bokeh.io.show(p)

In [50]:
def draw_bs_rep(data, func, rng):
    """Compute a bootstrap replicate from data."""
    bs_sample = rng.choice(data, size=len(data))
    return func(bs_sample)


# Replicates of each year
n_reps = 2000
bs_reps_1975 = np.array(
    [draw_bs_rep(bd_1975, np.mean, rng) for _ in range(n_reps)]
)
bs_reps_2012 = np.array(
    [draw_bs_rep(bd_2012, np.mean, rng) for _ in range(n_reps)]
)

# Get replicates of the difference
bd_reps_diff = bs_reps_2012 - bs_reps_1975

# Report the confidence interval
np.percentile(bd_reps_diff, [2.5, 97.5])

array([0.06421449, 0.39720874])

# Lesson 26: Dashboards

In [53]:
import pandas as pd
import numpy as np
import scipy.stats

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

notebook_url = 'localhost:8888'
bokeh.io.output_notebook()

Loading BokehJS ...

In [52]:
# Parameters; we'll start with standard Normal
mu = 0.0
sigma = 1.0

# Generate data
x = np.linspace(-10, 10, 200)
pdf = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

# Column data source for plot
source = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf))

# Build figure
p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=200,
    x_axis_label='x',
    y_axis_label='f(x)',
    x_range=[-10, 10],
)

# Put line on plot
p.line(source=source, x='x', y='pdf', line_width=2);

# We will not show it because if it is in a dashboard, a given plot can only
# be shown there in a notebook. Instead, it's displayed as an image below.

In [55]:
mu_slider = bokeh.models.Slider(title="µ", start=-5.0, end=5.0, step=0.1, value=0.0, width=100)
sigma_slider = bokeh.models.Slider(title="σ", start=0.1, end=5.0, step=0.1, value=1.0, width=100)

In [56]:
def norm_callback(attr, old, new):
    """Callback for updating data in Normal PDF plot."""
    # Pull the values off of each slider
    mu = mu_slider.value
    sigma = sigma_slider.value

    # Re-compute the y-values
    pdf = scipy.stats.norm.pdf(source.data['x'], loc=mu, scale=sigma)

    # Update the column data source
    source.data["pdf"] = pdf

In [57]:
mu_slider.on_change('value', norm_callback)
sigma_slider.on_change('value', norm_callback)

In [58]:
# Put the sliders one on top of the other
slider_layout = bokeh.layouts.column(
    bokeh.layouts.Spacer(height=30),
    mu_slider,
    bokeh.layouts.Spacer(height=15),
    sigma_slider,
)

# Put the sliders to the right of the plot
norm_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    slider_layout
)

In [59]:
def norm_app(doc):
    doc.add_root(norm_layout)

In [60]:
bokeh.io.show(norm_app, notebook_url=notebook_url)

In [61]:
def bohr_parameter(c, R, K, KdA, KdI, Kswitch):
    """Compute Bohr parameter based on MWC model."""
    # Big nasty argument of logarithm
    log_arg = (1 + c / KdA) ** 2 / (
        (1 + c / KdA) ** 2 + Kswitch * (1 + c / KdI) ** 2
    )

    return -np.log(R / K) - np.log(log_arg)


def fold_change(c, R, K, KdA, KdI, Kswitch):
    """Compute theoretical fold change for MWC model."""
    return 1 / (1 + np.exp(-bohr_parameter(c, R, K, KdA, KdI, Kswitch)))

In [62]:
sliders = dict(
    log_R_slider=bokeh.models.Slider(
        title="log₁₀ R (1/cell)", start=0, end=3, step=0.1, value=2
    ),
    log_K_slider=bokeh.models.Slider(
        title="log₁₀ K (1/cell)", start=-6, end=3, step=0.1, value=0
    ),
    log_KdA_slider=bokeh.models.Slider(
        title="log₁₀ KdA (1/mM)", start=-6, end=3, step=0.1, value=-2
    ),
    log_KdI_slider=bokeh.models.Slider(
        title="log₁₀ KdI (1/mM)", start=-6, end=3, step=0.1, value=-2
    ),
    log_Kswitch_slider=bokeh.models.Slider(
        title="log₁₀ Kswitch", start=-3, end=6, step=0.1, value=1,
    ),
)

In [63]:
# Concentration of inducer
c = np.logspace(-6, 2, 200)

# Take parameters from slider values
params = 10.0 ** np.array([slider.value for _, slider in sliders.items()])

# Fold change
fc = fold_change(c, *params)

# Data source
source = bokeh.models.ColumnDataSource(dict(c=c, fc=fc))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=350,
    x_axis_type="log",
    x_axis_label="[IPTG] (mM)",
    y_axis_label="fold change",
    x_range=[c.min(), c.max()],
    y_range=[-0.05, 1.05],
)

# Plot the curve
p.line(source=source, x="c", y="fc", line_width=2);

In [64]:
def induction_callback(attr, old, new):
    """Callback for updating induction plot."""
    # Take parameters from slider values
    params = 10.0 ** np.array([slider.value for _, slider in sliders.items()])

    # Update source
    source.data['fc'] = fold_change(source.data['c'], *params)


# Link the callback to the sliders
for _, slider in sliders.items():
    slider.on_change('value', induction_callback)

In [65]:
induction_layout = bokeh.layouts.row(
    p,
    bokeh.models.Spacer(width=15),
    bokeh.layouts.column(
        *[slider for _, slider in sliders.items()],
        width=200,
    ),
)

def induction_app(doc):
    doc.add_root(induction_layout)

bokeh.io.show(induction_app, notebook_url=notebook_url)

In [66]:
df = pd.read_csv('data/gfmt_sleep.csv', na_values='*')

# Add column for insomnia
df['insomnia'] = df['sci'] <= 16

df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence when incorrect hit,confidence when correct reject,confidence when incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True


In [67]:
# Options for x- and y- selector; omit part. num., gender, and insomnia
xy_options = list(
    df.columns[~df.columns.isin(["participant number", "gender", "insomnia"])]
)

In [68]:
x_selector = bokeh.models.Select(
    title="x", options=xy_options, value="percent correct", width=200,
)

y_selector = bokeh.models.Select(
    title="y", options=xy_options, value="confidence when correct", width=200,
)

colorby_selector = bokeh.models.Select(
    title="color by", options=["none", "gender", "insomnia",], value="none", width=200,
)

In [69]:
source = bokeh.models.ColumnDataSource(dict(x=df[x_selector.value], y=df[y_selector.value]))

# Add a column for colors; for now, all Bokeh's default blue
source.data['color'] = ['#1f77b4'] * len(df)

In [70]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
)

# Populate gylphs
circle = p.circle(source=source, x="x", y="y", color="color")

In [71]:
def gfmt_callback(attr, old, new):
    """Callback for updating plot of GMFT results."""
    # Update color column
    if colorby_selector.value == "none":
        source.data["color"] = ["#1f77b4"] * len(df)
    elif colorby_selector.value == "gender":
        source.data["color"] = [
            "#1f77b4" if gender == "f" else "#ff7e0e"
            for gender in df["gender"]
        ]
    elif colorby_selector.value == 'insomnia':
        source.data["color"] = [
            "#1f77b4" if insomnia else "#ff7e0e"
            for insomnia in df["insomnia"]
        ]

    # Update x-data and axis label
    source.data["x"] = df[x_selector.value]
    p.xaxis.axis_label = x_selector.value

    # Update x-data and axis label
    source.data["y"] = df[y_selector.value]
    p.yaxis.axis_label = y_selector.value

In [73]:
colorby_selector.on_change("value", gfmt_callback)
x_selector.on_change("value", gfmt_callback)
y_selector.on_change("value", gfmt_callback)

In [74]:
gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

def gfmt_app(doc):
    doc.add_root(gfmt_layout)

bokeh.io.show(gfmt_app, notebook_url=notebook_url)